In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as clr
import matplotlib.dates as md
import seaborn as sns
import datetime as datetime
import numpy as np
import os

%matplotlib inline

plt.style.use('default')

sns.set(style="whitegrid")

import warnings 
warnings.filterwarnings('ignore')

my_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [2]:
#CORRER PRIMERO EL NOTEBOOK FEATURES PARA CREAR LOS CSV DE LABELS Y FEATURES

labels = []
features_por_ventana = []

for i in range(0,5):
    labels.append(pd.read_csv('labels_v'+str(i+1)+'.csv'))
    labels[i].drop('Unnamed: 0',axis='columns',inplace=True)
    features_por_ventana.append(pd.read_csv('features_v'+str(i+1)+'.csv'))
    features_por_ventana[i].drop('Unnamed: 0',axis='columns',inplace=True)

In [3]:
labels[0].head()

,device_id,st,sc
0,1826643666390887030,0.015050,259200.000
1,7037174172278258682,0.029014,259200.000
2,3392065368947589877,0.057540,259200.000
3,1228982273563226229,0.126828,259200.000
4,4123059034628125459,0.132510,17800.664


In [4]:
features_por_ventana[0].head()

,device_id,cantidad_de_subastas_en_ventana,cantidad_subastas_Monday,cantidad_subastas_Tuesday,cantidad_subastas_Wednesday,cantidad_subastas_Thursday,cantidad_subastas_Friday,cantidad_subastas_Saturday,cantidad_subastas_Sunday,cantidad_de_clicks_en_ventana,...,cantidad_eventos_hora_15,cantidad_eventos_hora_16,cantidad_eventos_hora_17,cantidad_eventos_hora_18,cantidad_eventos_hora_19,cantidad_eventos_hora_20,cantidad_eventos_hora_21,cantidad_eventos_hora_22,cantidad_eventos_hora_23,device_os_version
0,41863526108385,35.0,0.0,0.0,0.0,0.0,1.0,34.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0
1,135153013040192,8.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,161514654074162,6.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,181891380775191,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,186034136943920,7.0,0.0,0.0,0.0,2.0,0.0,5.0,0.0,0.0,...,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Label

In [5]:
labels_b = []
tiempo_maximo = 259200

for i in range(0,5):
    labels[i] = (labels[i].loc[labels[i]['device_id'].isin(features_por_ventana[i]['device_id'])])[['device_id','st']]
    labels[i]['binary'] = np.where(labels[i]['st']>=tiempo_maximo,False,True)
    labels[i].sort_values(by='device_id',inplace=True)
    labels_b.append(labels[i][['binary','st']].to_records(index=False))

In [6]:
labels_b[0]

rec.array([( True, 157228.465866), ( True, 187854.009137),
           ( True,  10366.357746), ..., ( True,  50088.250389),
           ( True,  81157.046477), ( True,   1081.247579)],
          dtype=[('binary', '?'), ('st', '<f8')])

# Se agrega chequeo para que los df_feautures cumplan condiciones en los algoritmos a ser usados

In [7]:
for i in range(0,5):
    if (features_por_ventana[i].shape[0] != labels[i].shape[0]):
        raise AssertionError("Los df_features de cada ventana tienen que tener misma \
                cantidad de dispositivos(mismos device_id)")

# Algoritmo

In [8]:
from sksurv.preprocessing import OneHotEncoder

features_v1 = OneHotEncoder().fit_transform(features_por_ventana[0])
features_v2 = OneHotEncoder().fit_transform(features_por_ventana[1])
features_v1.head()

,device_id,cantidad_de_subastas_en_ventana,cantidad_subastas_Monday,cantidad_subastas_Tuesday,cantidad_subastas_Wednesday,cantidad_subastas_Thursday,cantidad_subastas_Friday,cantidad_subastas_Saturday,cantidad_subastas_Sunday,cantidad_de_clicks_en_ventana,...,cantidad_eventos_hora_15,cantidad_eventos_hora_16,cantidad_eventos_hora_17,cantidad_eventos_hora_18,cantidad_eventos_hora_19,cantidad_eventos_hora_20,cantidad_eventos_hora_21,cantidad_eventos_hora_22,cantidad_eventos_hora_23,device_os_version
0,41863526108385,35.0,0.0,0.0,0.0,0.0,1.0,34.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0
1,135153013040192,8.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,161514654074162,6.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,181891380775191,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,186034136943920,7.0,0.0,0.0,0.0,2.0,0.0,5.0,0.0,0.0,...,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
from sksurv.linear_model import CoxPHSurvivalAnalysis

estimator = CoxPHSurvivalAnalysis(alpha=0.0001)
estimator.fit(features_v1.head(10000),labels_b[0][:10000])

CoxPHSurvivalAnalysis(alpha=0.0001, n_iter=100, tol=1e-09, verbose=0)

In [11]:
pd.Series(estimator.coef_, index=features_v1.columns)

device_id                              -1.638405e-49
cantidad_de_subastas_en_ventana         1.426504e-32
cantidad_subastas_Monday                0.000000e+00
cantidad_subastas_Tuesday               0.000000e+00
cantidad_subastas_Wednesday             0.000000e+00
cantidad_subastas_Thursday              3.453876e-32
cantidad_subastas_Friday               -5.837218e-33
cantidad_subastas_Saturday             -1.445064e-32
cantidad_subastas_Sunday                0.000000e+00
cantidad_de_clicks_en_ventana           2.101467e-31
cantidad_clicks_Monday                  0.000000e+00
cantidad_clicks_Tuesday                 0.000000e+00
cantidad_clicks_Wednesday               0.000000e+00
cantidad_clicks_Thursday                5.802858e-31
cantidad_clicks_Friday                 -5.986919e-32
cantidad_clicks_Saturday               -3.102706e-31
cantidad_clicks_Sunday                  0.000000e+00
SUP_D                                   4.889128e-28
SUP_C                                   3.3292

#### El resultado es un vector de coeficientes, uno para cada variable, donde cada valor corresponde a la relación del registro y evento

In [12]:
estimator.score(features_v2.head(10000), labels_b[1][:10000])

0.5

# KNN

In [13]:
from sklearn.metrics import mean_squared_error

def root_mean_squared_error(y_test, predicted):
    return (np.sqrt(mean_squared_error(y_test,predicted)))

In [14]:
df = labels[0].sort_values(ascending=True,by='device_id').head(10000)
df.head()

,device_id,st,binary
257007,41863526108385,157228.465866,True
285766,135153013040192,187854.009137,True
70944,161514654074162,10366.357746,True
316713,181891380775191,256765.420614,True
155213,186034136943920,60166.331894,True


In [15]:
df_feat = features_v1.sort_values(ascending=True,by='device_id').head(10000)
df_feat.head()

,device_id,cantidad_de_subastas_en_ventana,cantidad_subastas_Monday,cantidad_subastas_Tuesday,cantidad_subastas_Wednesday,cantidad_subastas_Thursday,cantidad_subastas_Friday,cantidad_subastas_Saturday,cantidad_subastas_Sunday,cantidad_de_clicks_en_ventana,...,cantidad_eventos_hora_15,cantidad_eventos_hora_16,cantidad_eventos_hora_17,cantidad_eventos_hora_18,cantidad_eventos_hora_19,cantidad_eventos_hora_20,cantidad_eventos_hora_21,cantidad_eventos_hora_22,cantidad_eventos_hora_23,device_os_version
0,41863526108385,35.0,0.0,0.0,0.0,0.0,1.0,34.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0
1,135153013040192,8.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,161514654074162,6.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,181891380775191,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,186034136943920,7.0,0.0,0.0,0.0,2.0,0.0,5.0,0.0,0.0,...,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
from sklearn.model_selection import train_test_split

#TRAIN_TEST_SPLIT
#Split arrays or matrices into random train and test subsets
x_train, x_test, y_train, y_test = train_test_split(df_feat.drop(columns='device_id'),
                                                    df['st'],
                                                    test_size=0.2,
                                                    random_state=43)

In [17]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn import metrics

k=5 #-->Hiperparametro
neigh = KNeighborsRegressor(n_neighbors = k ,    #Cantidad de vecinos a usar --> K hiperparametro.
                            n_jobs=-1,           #Define la cantidad de procesadores a usar, -1 todos los disponibles.
                            weights='distance',  #Se define el peso como la inversa a la distancia, los mas cercanos tienen mayor influencia.
                            algorithm='brute'    #Fuerza bruta para el algorimo, podría ser KDTree o sino automatico.
                           )
#En weights se puede pasar una funcion definida por el usuario--> Segundo hiperparametro posible de KNN.

In [18]:
neigh.fit(x_train,y_train) #Fit the model using X as training data and Y as target values

KNeighborsRegressor(algorithm='brute', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
                    weights='distance')

In [19]:
predicted_knn = neigh.predict(x_test) #Predict the target for the provided data

In [20]:
root_mean_squared_error(y_test,predicted_knn)

56662.091346222325

In [21]:
neigh.score(x_train, y_train)

0.8144812144043072

In [22]:
neigh.score(x_test, y_test)

0.6506976161047189

# Gradient Boosting

In [23]:
from sklearn.ensemble import GradientBoostingRegressor

gradient_boost = GradientBoostingRegressor()

In [24]:
gradient_boost.fit(x_train,y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [25]:
predicted_gb = gradient_boost.predict(x_test)

In [26]:
gradient_boost.score(x_train, y_train)

0.9370502405795259

In [68]:
gradient_boost.score(x_test, y_test)

0.9351567280322869

In [28]:
root_mean_squared_error(y_test,predicted_gb)

24413.176200784415

In [63]:
from sklearn.model_selection import GridSearchCV
p_test1 = {'learning_rate':[0.15,0.1,0.05,0.01,0.005,0.001], 'n_estimators':[100,250,500,750,1000,1250,1500,1750]}

tuning = GridSearchCV(estimator =GradientBoostingRegressor(), param_grid = p_test1)

tuning.fit(df_feat.drop(columns='device_id'),df['st'])
tuning.best_params_, tuning.best_score_

({'learning_rate': 0.005, 'n_estimators': 1250}, 0.9335394614324747)

In [65]:
gradient_boost_2 = GradientBoostingRegressor(n_estimators= 1250, learning_rate=0.005)
gradient_boost_2.fit(x_train,y_train)
predicted_gb_2 = gradient_boost_2.predict(x_test)
gradient_boost_2.score(x_train, y_train)
root_mean_squared_error(y_test,predicted_gb_2)

24429.581248523762

In [67]:
gradient_boost_2.score(x_test, y_test)
#Baja

0.9350695526174552

In [71]:
p_test2 = {'max_depth':[2,3,4,5,6,7]}
tuning = GridSearchCV(estimator =GradientBoostingRegressor(), param_grid = p_test2)

tuning.fit(df_feat.drop(columns='device_id'),df['st'])
tuning.best_params_, tuning.best_score_

({'max_depth': 3}, 0.9338475449027231)

In [72]:
gradient_boost_3 = GradientBoostingRegressor(max_depth= 3)
gradient_boost_3.fit(x_train,y_train)
gradient_boost_3.score(x_test, y_test)
#baja

0.9351380720519836

In [73]:
p_test3 = {'min_samples_split':[2,4,6,8,10,20,40,60,100], 'min_samples_leaf':[1,3,5,7,9] }

tuning = GridSearchCV(estimator =GradientBoostingRegressor(), param_grid = p_test3)

tuning.fit(df_feat.drop(columns='device_id'),df['st'])
tuning.best_params_, tuning.best_score_

({'min_samples_leaf': 5, 'min_samples_split': 60}, 0.9339450691357554)

In [74]:
gradient_boost_4 = GradientBoostingRegressor(min_samples_leaf= 5, min_samples_split= 60)
gradient_boost_4.fit(x_train,y_train)
gradient_boost_4.score(x_test, y_test)
#Baja

0.9350079834773051

# Random Forest

In [29]:
from sklearn.ensemble import RandomForestRegressor

random_forest = RandomForestRegressor()

In [30]:
random_forest.fit(x_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [31]:
predicted_rf = random_forest.predict(x_test)

In [32]:
root_mean_squared_error(y_test,predicted_rf)

26603.45605627471

In [33]:
random_forest.score(x_train, y_train)

0.9637152326553651

In [34]:
random_forest.score(x_test,y_test)

0.9229996908506215

# Linear Regression

In [35]:
from sklearn.linear_model import LinearRegression

reg_lineal = LinearRegression()

In [36]:
reg_lineal.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [37]:
reg_lineal.score(x_train,y_train)

0.00725428784723281

In [38]:
reg_lineal.score(x_test,y_test)

0.011756279074781273

# Ada Boost

In [39]:
from sklearn.ensemble import AdaBoostRegressor

ada_boost = AdaBoostRegressor()

In [40]:
ada_boost.fit(x_train,y_train)

AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='linear',
                  n_estimators=50, random_state=None)

In [41]:
predicted_ab = ada_boost.predict(x_test)

In [48]:
root_mean_squared_error(y_test,predicted_ab)

24793.306589140786

In [49]:
ada_boost.score(x_train, y_train)

0.9324363488726378

In [50]:
ada_boost.score(x_test,y_test)

0.9331216959394626

# Decision Tree

In [42]:
from sklearn.tree import DecisionTreeRegressor

arbol_dec = DecisionTreeRegressor()

In [43]:
arbol_dec.fit(x_train,y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [53]:
predicted_dt = arbol_dec.predict(x_test) 

In [54]:
root_mean_squared_error(y_test,predicted_dt)

31685.815430226838

In [55]:
arbol_dec.score(x_train,y_train)

0.9708939774401235

In [56]:
arbol_dec.score(x_test,y_test)

0.8907689455636174

In [57]:
feature_importances = pd.DataFrame(arbol_dec.feature_importances_,
                                   index = x_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
feature_importances.head(10)

,importance
cantidad_subastas_Thursday,0.778212
cantidad_subastas_Friday,0.173654
cantidad_subastas_Saturday,0.023967
cantidad_de_subastas_en_ventana,0.008015
cantidad_de_eventos_en_ventana,0.001236
aplicacion_mas_usada,0.001119
tipo_evento_mas_frec,0.001060
Porc_I-E,0.000908
cantidad_instalaciones_Thursday,0.000735
cantidad_eventos_Friday,0.000727


# XGBoost

In [174]:
from xgboost import XGBRegressor

xgboost = XGBRegressor()

In [175]:
xgboost.fit(x_train,y_train)

[15:26:02] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [180]:
predicted_xg = xgboost.predict(x_test) 
x_test

,cantidad_de_subastas_en_ventana,cantidad_subastas_Monday,cantidad_subastas_Tuesday,cantidad_subastas_Wednesday,cantidad_subastas_Thursday,cantidad_subastas_Friday,cantidad_subastas_Saturday,cantidad_subastas_Sunday,cantidad_de_clicks_en_ventana,cantidad_clicks_Monday,...,cantidad_eventos_hora_15,cantidad_eventos_hora_16,cantidad_eventos_hora_17,cantidad_eventos_hora_18,cantidad_eventos_hora_19,cantidad_eventos_hora_20,cantidad_eventos_hora_21,cantidad_eventos_hora_22,cantidad_eventos_hora_23,device_os_version
9415,425.0,0.0,0.0,0.0,243.0,175.0,7.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00
6377,169.0,0.0,0.0,0.0,76.0,67.0,26.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00
8019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.000000e+00
7754,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8.840146e+18
4961,5.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00
7364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,5.908703e+17
5316,105.0,0.0,0.0,0.0,29.0,29.0,47.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00
4265,15.0,0.0,0.0,0.0,4.0,1.0,10.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00
6302,16.0,0.0,0.0,0.0,0.0,7.0,9.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00
8581,33.0,0.0,0.0,0.0,20.0,13.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,23.0,0.0,0.0,3.0,0.0,0.0,0.000000e+00


In [179]:
y_test

149910     57449.696334
4043         141.648819
373166    259200.000000
278965    180996.490733
286526    188599.270764
321969    259200.000000
50201       6288.060023
55823       7319.611165
209375     93646.066554
177484     72271.171842
163501     64459.925096
345203    259200.000000
182            2.548292
293615    202516.676281
239754    137908.000140
348091    259200.000000
41081       4661.068962
304543    237327.594551
157999     61581.359546
242012    140756.404899
267585    169014.889910
183128     75755.747178
176988     71969.142852
342263    259200.000000
278807    180831.118715
252377    152304.739615
17136       1258.600325
84130      13228.976827
178053     72610.273347
385467    259200.000000
              ...      
255290    155381.830107
232747    111962.231782
142558     53555.222706
185576     77330.529584
319872    259200.000000
110739     20045.604850
215891     97861.449397
140894     52664.387738
117818     23528.761954
214896     97233.759723
3007          91

In [177]:
root_mean_squared_error(y_test,predicted_xg)

24448.11338753524

In [162]:
xgboost.score(x_train,y_train)

0.9370048320419184

In [163]:
xgboost.score(x_test,y_test)

0.934971003528712

In [93]:
parameters_xgb = {
        'num_boost_round': [100, 250, 500],
        'eta': [0.05, 0.1, 0.3],
        'max_depth': [6, 9, 12],
        'subsample': [0.9, 1.0],
        'colsample_bytree': [0.9, 1.0],
}

tuning_xgb = GridSearchCV(estimator = XGBRegressor(), param_grid = parameters_xgb)

tuning_xgb.fit(df_feat.drop(columns='device_id'),df['st'])
tuning_xgb.best_params_, tuning.best_score_

[14:44:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:46] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:49] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:51] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:54] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:56] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:59] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:45:01] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:45:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

({'colsample_bytree': 0.9,
  'eta': 0.05,
  'max_depth': 6,
  'num_boost_round': 100,
  'subsample': 1.0},
 0.9339450691357554)

In [ ]:
'colsample_bytree': 1.0,
  'eta': 0.05,
  'max_depth': 6,
  'num_boost_round': 100,
  'subsample': 1.0

# GradientBoostingSurvivalAnalysis

In [64]:
from sksurv.ensemble import GradientBoostingSurvivalAnalysis

model = GradientBoostingSurvivalAnalysis()

In [ ]:
model.fit(features_v1.head(5000),labels_b[0][:5000])

In [60]:
model.score(features_v2.head(5000),labels_b[1][:5000])

0.8629979777886098

# Majority Voting

In [46]:
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import SGDRegressor
from sklearn import model_selection

In [59]:
#Jugando con cuales se usan se obtienen distintos resultados
estimators = []
estimators.append(('RandomForest',RandomForestRegressor()))
estimators.append(('DecisionTree', DecisionTreeRegressor()))
estimators.append(('AdaBoost', AdaBoostRegressor()))
#estimators.append(('SGDRegressor',SGDRegressor()))
estimators.append(('XGBoost',XGBRegressor()))
estimators.append(('GradientBoosting',GradientBoostingRegressor()))

In [60]:
voting = VotingRegressor(estimators)
voting.fit(x_train,y_train)
#results = model_selection.cross_val_score(voting,x_train,y_train)
#results
voting.score(x_test,y_test)


[23:36:23] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.9318587690580298

# Prediccion

1000169251625791246
sc


XGBoostError: [15:24:51] src/io/local_filesys.cc:86: LocalFileSystem.GetPathInfo: 1000169251625791246 error: No such file or directory
Stack trace:
  [bt] (0) /home/gonzalo/miniconda3/lib/libxgboost.so(dmlc::io::LocalFileSystem::GetPathInfo(dmlc::io::URI const&)+0x416) [0x7fe53178ade6]
  [bt] (1) /home/gonzalo/miniconda3/lib/libxgboost.so(dmlc::io::InputSplitBase::InitInputFileInfo(std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&, bool)+0x109) [0x7fe5317e4be9]
  [bt] (2) /home/gonzalo/miniconda3/lib/libxgboost.so(dmlc::io::InputSplitBase::Init(dmlc::io::FileSystem*, char const*, unsigned long, bool)+0x77) [0x7fe5317e51c7]
  [bt] (3) /home/gonzalo/miniconda3/lib/libxgboost.so(dmlc::InputSplit::Create(char const*, char const*, unsigned int, unsigned int, char const*, bool, int, unsigned long, bool)+0xd89) [0x7fe531789c89]
  [bt] (4) /home/gonzalo/miniconda3/lib/libxgboost.so(dmlc::InputSplit::Create(char const*, unsigned int, unsigned int, char const